## Deep Learning with Python

Let's put all the pieces together: Python, NumPy/pandas/Matplotlib, Linear Algebra and Neural Networks, and learn PyTorch. PyTorch is an open source library for Python easily used to train neural networks.



In [1]:
import torch.nn as nn
import inspect

# Using dir()
print("Using dir():")
print(dir(nn.Module))

# Using inspect.getmembers()
print("\nUsing inspect.getmembers():")
methods = inspect.getmembers(nn.Module, predicate=inspect.isfunction)
for method in methods:
    print(method[0])

# Using help()
print("\nUsing help():")



Using dir():
['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_call_impl', '_compiled_call_impl', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_load_from_state_dict', '_maybe_warn_non_full_backward_hook', '_named_members', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict', '_slow_forward', '_version', '_wrapped_call_impl', 'add_module', 'apply', 'bfloat16', 'buffers', 'call_super_init', 'children', 'compile', 'cpu', 'cuda', 'double', 'dump_patches', 'eval', 'extra_repr', 'float', 'forward', 'get_buffer'

## Tensors 

The fundamental data structure of the pytorch is `Tensor`, Our neural networks are just the linear algebra computations on these tensors.

- vector is a one dimensional tensor
- a matrix is a 2 - dimensional tensor
- an array with 3 indices is a 3 dimensional tensor (Example RGB images) 

In [2]:
import numpy as np
import torch 

In [3]:
x = torch.rand(3, 2)
x

tensor([[0.7086, 0.3793],
        [0.4785, 0.4835],
        [0.5868, 0.5180]])

In [4]:
x.size()

torch.Size([3, 2])

In [5]:
y = torch.ones(x.size()) # it will calculate the size of x and pass it to the y
y

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [6]:
z = x + y

z

tensor([[1.7086, 1.3793],
        [1.4785, 1.4835],
        [1.5868, 1.5180]])

In [7]:
z[0] # Get the first row

tensor([1.7086, 1.3793])

In [8]:
z[:, 1:] # All the rows for the 2nd column 

tensor([[1.3793],
        [1.4835],
        [1.5180]])

In [9]:
z[:, :1] # All the rows for the 1st column 

tensor([[1.7086],
        [1.4785],
        [1.5868]])

In [10]:
z.add(1) # It will change the z value temparaly means not inplace

tensor([[2.7086, 2.3793],
        [2.4785, 2.4835],
        [2.5868, 2.5180]])

In [11]:
z # The value did not change as you can see here 

tensor([[1.7086, 1.3793],
        [1.4785, 1.4835],
        [1.5868, 1.5180]])

In [12]:
z.add_(1) # It will 

tensor([[2.7086, 2.3793],
        [2.4785, 2.4835],
        [2.5868, 2.5180]])

In [13]:
z # Tha value chnaged completly

tensor([[2.7086, 2.3793],
        [2.4785, 2.4835],
        [2.5868, 2.5180]])

## Reshaping

Reshaping the tensors is usually common operation. First to get the size of tensor use `.size()`. Then reshape a tensor using `.resize()` or `.resize_()` notice the underscore, reshaping an in-place operation

In [22]:
z1 = z.resize(2, 3)
z1

/Users/mekalathuruchenchaiah/Desktop/PROGRAMMING/Projects/GPU-pytorch/env/lib/python3.8/site-packages/torch/_tensor.py:836: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


tensor([[2.7086, 2.3793, 2.4785],
        [2.4835, 2.5868, 2.5180]])

In [19]:
z # Not chnaged permaneltly 

tensor([[2.7086, 2.3793],
        [2.4785, 2.4835],
        [2.5868, 2.5180]])

In [23]:
z.resize_(3, 2)

tensor([[2.7086, 2.3793],
        [2.4785, 2.4835],
        [2.5868, 2.5180]])

In [25]:
z # Now changed permanently

tensor([[2.7086, 2.3793],
        [2.4785, 2.4835],
        [2.5868, 2.5180]])

## Numpy to Torch and back

Converting between Numpy arrays and Torch tensors is super simple and useful. To create a tensor from a Numpy array, use torch `.from _numpy()` . To convert a tensor to a Numpy array, use the `.numpy()` method.

In [27]:
a = np.random.rand(3, 4)
a

array([[0.56016318, 0.09058812, 0.69882777, 0.34597718],
       [0.94429734, 0.96116969, 0.20071813, 0.20311369],
       [0.0412566 , 0.25702409, 0.04257092, 0.12484024]])

In [34]:
b = torch.from_numpy(a) # Changes the array to tensor 
b 

tensor([[0.5602, 0.0906, 0.6988, 0.3460],
        [0.9443, 0.9612, 0.2007, 0.2031],
        [0.0413, 0.2570, 0.0426, 0.1248]], dtype=torch.float64)

In [35]:
b.numpy() # Will change the tensor to numpy array

array([[0.56016318, 0.09058812, 0.69882777, 0.34597718],
       [0.94429734, 0.96116969, 0.20071813, 0.20311369],
       [0.0412566 , 0.25702409, 0.04257092, 0.12484024]])

## Defining Networks

Let's see how to build neural networks using the pytorch 



In [14]:
import numpy as np
import torch 

import helper 

import matplotlib.pyplot as plt 
from torchvision import datasets, transforms

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

The dataset we used for this example is `MNIST`. This dataset contains the bunch of handdrawn digits from 0 through 9 which is used to classify the images into digits. Here we are using the `torchvision` to download and load the training data.

In [15]:
# Define a transform to normalize the data 

transform = transforms.Compose([transforms.ToTensor(), 
                               transforms.Normalize(0.5, 0.5, 0.5), (0.5, 0.5, 0.5),])


# 
